# Limpeza e Preparação de Dados - Detecção de Phishing

Este notebook realiza a limpeza e preparação do dataset de URLs de phishing para treinamento de modelos de Machine Learning.

## Importação de Bibliotecas

Importamos as bibliotecas necessárias para manipulação de dados e download do dataset.

In [5]:
import shutil

import kagglehub
import pandas as pd
import os

## Download do Dataset

O dataset utilizado é o **Phishing Site URLs** do Kaggle, que contém URLs classificadas como phishing ou legítimas.

Se o dataset não existir localmente, ele será baixado automaticamente usando a API do Kaggle Hub.

In [6]:
dataset_path = "data/phishing_site_urls_cleaned.csv"

if not os.path.exists(dataset_path):
    downloaded_dataset_path = kagglehub.dataset_download("taruntiwarihp/phishing-site-urls")

    shutil.copytree(downloaded_dataset_path, "data", dirs_exist_ok=True)

    print("Dataset downloaded successfully")

## Carregamento e Renomeação de Colunas

Carregamos o dataset e padronizamos os nomes das colunas para facilitar o processamento:
- Todas as colunas são convertidas para **lowercase**
- A coluna `label` é renomeada para `is_phishing` para maior clareza semântica

In [7]:
df = pd.read_csv("data/phishing_site_urls.csv")

# rename every column to lowercase
df.columns = df.columns.str.lower()

df.rename(columns={"label": "is_phishing"}, inplace=True)
df.head()

,url,is_phishing
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad


## Engenharia de Features

Criamos diversas features (características) extraídas das URLs para alimentar o modelo de Machine Learning.

### Categorias de Features:

| Categoria | Features | Descrição |
|-----------|----------|-----------|
| **Tamanho** | `url_length`, `domain_length` | Comprimento da URL e do domínio |
| **Caracteres especiais** | `dot_count`, `hyphen_count`, `underscore_count`, `slash_count`, `question_count`, `equal_count`, `at_count` | Contagem de caracteres especiais |
| **Composição** | `digits_count`, `letters_count`, `digit_letter_ratio` | Análise de dígitos e letras |
| **Indicadores** | `has_ip`, `has_https`, `is_shortened` | Presença de IP, HTTPS ou encurtadores |
| **Palavras suspeitas** | `suspicious_words` | Contagem de palavras associadas a phishing |

### Palavras Suspeitas

Definimos um conjunto de palavras frequentemente encontradas em URLs de phishing, incluindo:
- Termos de **autenticação** (login, signin, password)
- Palavras relacionadas a **bancos e finanças** (bank, payment, crypto)
- Expressões de **urgência** (urgent, alert, limited)
- Nomes de **serviços populares** (google, facebook, netflix)
- Palavras em **português** relacionadas a golpes (aposta, dinheiro, ganhar)

In [8]:
suspicious_words = {
    # Autenticação & Conta
    'login', 'signin', 'signup', 'register', 'password', 'credential',
    'verification', 'authenticate', 'account', 'profile', 'settings',
    'recover', 'reset', 'confirm', 'validation',

    # Banco & Finanças
    'bank', 'secure', 'wallet', 'payment', 'pay', 'billing', 'invoice',
    'paypal', 'stripe', 'visa', 'mastercard', 'crypto', 'bitcoin',
    'finance', 'transfer', 'withdraw', 'deposit',

    # Urgência & Ação
    'update', 'required', 'action', 'urgent', 'alert', 'notification',
    'limited', 'access', 'suspended', 'locked', 'security', 'verify',
    'now', 'free', 'win', 'prize',

    # Serviços/Plataformas populares
    'google', 'icloud', 'apple', 'microsoft', 'facebook', 'instagram',
    'whatsapp', 'netflix', 'twitter', 'linkedin', 'youtube', 'tiktok',
    'pinterest', 'reddit', 'github', 'gitlab', 'bitbucket',

    # Termos administrativos
    'webmaster', 'admin', 'root', 'support', 'service', 'help', 'contact',
    'about', 'terms', 'privacy',

    # Frases de “click here”
    # (apenas a base; os complementos são tratados em tempo de execução)
    'click here',

    # Domínios encurtadores
    'bit.ly', 'tinyurl.com', 'goo.gl', 'bitly.com', 'tiny.cc', 'bit.do',

    # Subdomínios suspeitos (normalizados, sem duplicatas)
    'app', 'mail', 'secure', 'login', 'signin', 'signup', 'register',
    'password', 'credential', 'verification', 'authenticate', 'account',
    'profile', 'settings', 'recover', 'reset', 'confirm', 'validation',

    # Palavras em português (normalizadas)
    'aposta', 'dinheiro', 'ganhar', 'milionario', 'sorte', 'lucro',
    'riqueza', 'sucesso', 'vida', 'amor', 'saude', 'familia', 'trabalho',
    'carreira', 'loteria', 'lotofacil', 'lotomania', 'megasena', 'quina',
    'timemania', 'dupla-sena', 'loteca', 'loterica', 'loterij',
}


features_required = [
    'url_length', 'domain_length',
    'dot_count', 'hyphen_count', 'underscore_count', 'slash_count',
    'question_count', 'equal_count', 'at_count',
    'digits_count', 'letters_count',
    'has_ip', 'has_https',
    'digit_letter_ratio', 'suspicious_words', 'is_shortened'
]

df["is_phishing"] = df["is_phishing"].replace({"bad": 1, "good": 0})
df['has_ip'] = df['url'].str.contains(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
df['url_length'] = df['url'].str.len()
df['has_at'] = df['url'].str.contains(r'@')
df['dot_count'] = df['url'].apply(lambda x: str(x).count('.'))
df['num_digits'] = df['url'].apply(lambda x: sum(c.isdigit() for c in x))
df['domain_length'] = df['url'].str.split('/').str[2].str.len()
df['hyphen_count'] = df['url'].apply(lambda x: str(x).count('-'))
df['underscore_count'] = df['url'].apply(lambda x: str(x).count('_'))
df['slash_count'] = df['url'].apply(lambda x: str(x).count('/'))
df['question_count'] = df['url'].apply(lambda x: str(x).count('?'))
df['equal_count'] = df['url'].apply(lambda x: str(x).count('='))
df['at_count'] = df['url'].apply(lambda x: str(x).count('@'))
df['digits_count'] = df['url'].apply(lambda x: sum(c.isdigit() for c in x))
df['letters_count'] = df['url'].apply(lambda x: sum(c.isalpha() for c in x))
df['digit_letter_ratio'] = df['digits_count'] / df['letters_count']
df['suspicious_words'] = df['url'].apply(lambda x: sum(word in x for word in suspicious_words))
df['is_shortened'] = df['url'].apply(lambda x: 1 if 'bit.ly' in x or 'tinyurl' in x else 0)

df.head()

C:\Users\GABRIEL\AppData\Local\Temp\ipykernel_12116\1376777085.py:55: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["is_phishing"] = df["is_phishing"].replace({"bad": 1, "good": 0})


,url,is_phishing,has_ip,url_length,has_at,dot_count,num_digits,domain_length,hyphen_count,underscore_count,slash_count,question_count,equal_count,at_count,digits_count,letters_count,digit_letter_ratio,suspicious_words,is_shortened
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,1,False,225,False,6,58,15.0,4,4,10,1,4,0,58,135,0.429630,4,0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,1,False,81,False,5,1,9.0,2,1,4,0,2,0,1,65,0.015385,2,0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,1,False,177,False,7,47,4.0,1,0,11,0,0,0,47,111,0.423423,4,0
3,mail.printakid.com/www.online.americanexpress....,1,False,60,False,6,0,10.0,0,0,2,0,0,0,0,52,0.000000,1,0
4,thewhiskeydregs.com/wp-content/themes/widescre...,1,False,116,False,1,21,6.0,1,0,10,1,0,0,21,82,0.256098,0,0


## Limpeza de Dados

Removemos registros problemáticos para garantir a qualidade do dataset:

1. **Duplicatas**: URLs repetidas são removidas para evitar viés no treinamento
2. **Valores nulos**: Registros com URL ou label ausentes são descartados

In [11]:
# Clean the dataframe
df = df.drop_duplicates(subset=['url'])

# Remove rows with null values
df = df.dropna(subset=['url'])
df = df.dropna(subset=['is_phishing'])

### Por que removemos os registros?

Como se trata de dados "crus" (apenas a URL), não existe informação suficiente para determinar com segurança o valor de uma célula ausente ou ambígua apenas pela string da URL. Por isso removemos registros problemáticos, URLs malformadas, duplicatas e linhas com valores nulos em features essenciais, para reduzir ruído e viés no conjunto de treino. Se quisermos recuperar ou imputar esses valores faltantes, isso deverá ser feito via um modelo treinado que estime as informações ausentes a partir das demais features; portanto precisaremos desenvolver/aplicar esse modelo para realizar tais inferências.

## Exportação do Dataset

O dataset limpo e enriquecido com as novas features é salvo em um novo arquivo CSV para uso posterior no treinamento do modelo.

In [10]:
# write the dataframe to a csv file
df.to_csv("data/phishing_site_urls_cleaned.csv", index=False)

---

## Resumo

O pipeline de limpeza realizou as seguintes operações:

- Download automático do dataset do Kaggle
- Padronização dos nomes das colunas
- Criação de **15+ features** baseadas nas características das URLs
- Remoção de duplicatas e valores nulos
- Exportação do dataset processado

O arquivo `phishing_site_urls_cleaned.csv` está pronto para ser utilizado no treinamento de modelos de detecção de phishing.